In [ ]:
import keras 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from  sklearn.metrics import confusion_matrix


from keras.preprocessing import image
from keras import layers 
from keras import models

In [ ]:
def d4_images():
    base_dir = '/kaggle/input/lumbartesttrain'
    train_dir = os.path.join(base_dir, 'Train_D4')
    validation_dir = os.path.join(base_dir, 'Test_D4')
    return  train_dir, validation_dir

In [ ]:
def all_images():
    base_dir = '/kaggle/input/total-images/archive/archive'
    train_dir = os.path.join(base_dir, 'Train')
    validation_dir = os.path.join(base_dir, 'Validation')
    return  train_dir, validation_dir

In [ ]:
def d4_seg_images():
    base_dir = '/kaggle/input/d4-segmented-images/Test_Train_D4'
    train_dir = os.path.join(base_dir, 'Train_D4')
    validation_dir = os.path.join(base_dir, 'Test_D4')
    return  train_dir, validation_dir

In [ ]:
#change   function   depending   on   whether  all   photos  are   used  or  just   some
train_dir  =  d4_seg_images()[0]
validation_dir  =  d4_seg_images()[1]

In [ ]:
train_datagen =keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255,
    rotation_range=0.5,
    width_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.2,
    horizontal_flip=True,
    #samplewise_center=True,
    #samplewise_std_normalization=True
)

val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(320, 320), 
        batch_size=4,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(320, 320),
        batch_size=4,
        class_mode='binary',
    shuffle=False)



In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
imgs, labels = next(train_generator)

def plotImages(images_arr):
    fig, axes = plt.subplots(1,5, figsize = (20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()
    
plotImages(imgs)
print(labels)


In [ ]:
img_input = layers.Input(shape=(320, 320, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)
x = layers.BatchNormalization()(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.BatchNormalization()(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.BatchNormalization()(x)
x=layers.Dropout(0.1)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.BatchNormalization()(x)
x=layers.Dropout(0.1)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.BatchNormalization()(x)
x=layers.Dropout(0.2)(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(img_input, output)
model.summary()


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [ ]:
#weight class calculation
def class_weights(image_type):
    if image_type == "D4":
        n_samples= 385   
        n_classes= 2 
        n_sample0= 135 
        n_sample1= 250
    elif image_type == "All":
        n_samples= 752   
        n_classes= 2 
        n_sample0= 345 
        n_sample1= 407
    elif image_type == "Seg":
        n_samples= 768   
        n_classes= 2 
        n_sample0= 352 
        n_sample1= 416
    

    #Weights for class 0 (healthy):
    w0=  n_samples/(n_classes*n_sample0) 
    #Weights for class 1 (unhealthy):
    w1= n_samples/(n_classes*n_sample1)
    
    return w0,w1
w0 =class_weights("Seg")[0]
w1= class_weights("Seg")[1]

class_weights = {0:w0, 1: w1}

print(class_weights)

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=20,  
      epochs=95,
      validation_data=validation_generator,
      validation_steps=4,  
      verbose=2,
    class_weight = class_weights)

In [ ]:
def  produce_graph():
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()
    
produce_graph()

In [ ]:
#evaluation = model.evaluate(train_generator)
#print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

#evaluation = model.evaluate(validation_generator)
#print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

from sklearn.metrics import classification_report, confusion_matrix

num_of_test_samples = 96
batch_size = 4


Y_pred = model.predict(validation_generator, num_of_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
confu =confusion_matrix(validation_generator.classes, y_pred)
print(confu)
print('Classification Report')
target_names = ['Healthy', 'Unhealthy']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
import numpy as np
y_pred = ann.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

In [ ]:
sns.heatmap(confu, annot=True, fmt="d")

In [ ]:
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')



In [ ]:
test_predictions_baseline = model.predict(test_features, batch_size=96)

plot_cm(test_labels, test_predictions_baseline)